In [1]:
import copy
import numpy as np
import os
from utils import check_paths

import mne
from mne.stats import permutation_cluster_1samp_test
from mne.stats import permutation_t_test

import scipy
from scipy.stats import zscore
from scipy.sparse import coo_matrix, save_npz

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
from matplotlib.cm import ScalarMappable

%matplotlib qt

# Figure 4
DONE

In [2]:
analysis_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group'
power_data = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"Y_ALL_subs_BL_go_power-tfr.h5"))
power_data

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y group\Y_ALL_subs_BL_go_power-tfr.h5 ...


Data type,Average Power Estimates
Data source,Epochs
Number of epochs,60
Dims,"channel, freq, time"
Estimation method,morlet
Number of channels,60
Number of timepoints,167
Number of frequency bins,40
Frequency range,4.00 – 80.00 Hz


In [3]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
analysis_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group'
tfr_group_dir = os.path.join(analysis_dir, 'sensors', 'tfr')
paper1_figs_save_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group\\paper1 figs'
check_paths(tfr_group_dir, paper1_figs_save_path)

group = 'Y'
sub_name = '_ALL_SUBS'
tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go'] # '_plan' for PLANNING or '_go' for EXECUTION

subs_dir = os.path.join(eeg_data_dir, group)

In [ ]:
# Save stats for each condition
for task in tasks:
    for task_stage in task_stages:
        if task == '_BL':
            block_names = ['']
        else:
            block_names = ['_baseline', '_adaptation']
        for block_name in block_names:
            power_data = []
            itc_data = []

            for sub_name in os.listdir(subs_dir):

                preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
                analysis_dir = os.path.join(preproc_dir, 'analysis')    

                # Read TFR and ITC data
                power = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"))
                itc = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"))

                power_data.append(power.get_data()) # power is a list with one element, so we take the first element
                # itc_zscore = zscore(itc.get_data(), axis=0, nan_policy='omit') # zscore test - p-values too high
                itc_data.append(itc.get_data()) # itc is a list with one element

            ### POWER TFR  STATS ###
            power_data = np.array(power_data) # shape (n_subjects, n_channels, n_freqs, n_times)
            np.save(os.path.join(tfr_group_dir, f"TFR_CONCATENATED_{group}{task}{task_stage}{block_name}.npy"), power_data)
            itc_data = np.array(itc_data) # shape (n_subjects, n_channels, n_freqs, n_times)
            np.save(os.path.join(tfr_group_dir, f"ITC_CONCATENATED_{group}{task}{task_stage}{block_name}.npy"), itc_data)
            print(power_data.shape, itc_data.shape)

            ### POWER TFR  STATS ###
            power_data_ave = np.mean(power_data, axis=(0,1)) # average across subs and channels
            # Average across channels but keep subjects
            power_data_subs = np.mean(power_data, axis=1)  # shape (n_subjects, n_freqs, n_times)
            # Reshape to (n_subjects, n_freqs * n_times)
            X_power = power_data_subs.reshape(power_data_subs.shape[0], -1)
            # Run permutation t-test across subjects
            T0_pow, p_values_pow, H0_pow = permutation_t_test(X_power, n_permutations=100000, tail=0, n_jobs=None) # two-tailed
            # Reshape p_values back to (n_freqs, n_times) 
            p_values_pow = p_values_pow.reshape(power_data_subs.shape[1], power_data_subs.shape[2])

            # Save the results
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_T_obs.npy"), T0_pow)
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_p_values.npy"), p_values_pow)
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_H0.npy"), H0_pow)

            ### ITC TFR  STATS ###
            # Shape before averaging = (n_subjects, n_channels, n_freqs, n_times)
            itc_data_ave = np.mean(itc_data, axis=(0,1)) # average across subs and channels
            # Average across channels but keep subjects
            itc_data_subs = np.mean(itc_data, axis=1)  # shape (n_subjects, n_freqs, n_times)
            # Reshape to (n_subjects, n_freqs * n_times)
            X_itc = itc_data_subs.reshape(itc_data_subs.shape[0], -1)
            # Run permutation t-test across subjects
            T0_itc, p_values_itc, H0_itc = permutation_t_test(X_itc, n_permutations=100000, tail=1, n_jobs=None) # one-tailed (greater than 0)
            # Reshape p_values back to (n_freqs, n_times)
            p_values_itc = p_values_itc.reshape(itc_data_subs.shape[1], itc_data_subs.shape[2])

            # Save the results
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_T_obs.npy"), T0_itc)
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_p_values.npy"), p_values_itc)
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_H0.npy"), H0_itc)




In [8]:
def plot_tfr_stat(ax, times, freqs, data_ave, p_values, alpha, cmap, clim):
    """Plot TFR with nonsignificant in gray, significant in color overlay."""

    max_dims = (40, 167)
    if data_ave.shape[0] > max_dims[0] or data_ave.shape[1] > max_dims[1]:
        print(f"Data shape {data_ave.shape} exceeds max dims {max_dims}. Truncating.")
        data_ave = data_ave[:max_dims[0], :max_dims[1]]
        p_values = p_values[:max_dims[0], :max_dims[1]]
        freqs = freqs[:max_dims[0]]
        times = times[:max_dims[1]]

    # Build significance mask
    significant_mask = p_values < alpha
    masked_data = np.ma.masked_where(~significant_mask, data_ave)

    print(f"data shape: {data_ave.shape}, p_values shape: {p_values.shape}")
    print(f"times shape: {times.shape}, freqs shape: {freqs.shape}")
    print(f"Masked data shape: {masked_data.shape}, significant mask shape: {significant_mask.shape}")

    # Adjust ax position to make room for colorbar
    shrink = 1 # modify shrinking factor if needed
    pos = ax.get_position()
    x0, y0, x1, y1 = pos.x0, pos.y0, pos.x1, pos.y1
    w, h = x1 - x0, y1 - y0
    new_w, new_h = w * shrink, h * shrink
    x0_new = x0 + (w - new_w) / 2
    y0_new = y0 + (h - new_h) / 2
    ax.set_position([x0_new, y0_new, new_w, new_h])

    # Base in gray
    ax.pcolormesh(times, freqs, data_ave, cmap="gray", shading="auto")
    # Overlay significant in color
    im = ax.pcolormesh(times, freqs, masked_data, cmap=cmap, shading="auto",
                       vmin=clim[0], vmax=clim[1])

    ax.set_xlabel("Time (s)", fontsize=6)
    ax.set_ylabel("Frequency (Hz)", fontsize=6)
    ax.set_yscale("log")

    # pick 10 evenly spaced freqs
    yticks = np.linspace(0, len(freqs) - 1, 10, dtype=int)
    ytick_vals = freqs[yticks]
    ax.set_yticks(ytick_vals)
    ax.set_yticklabels([f"{f:.2f}" for f in ytick_vals])
    ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
    ax.tick_params(axis='both', which='major', left=False, labelsize=6)
    ax.tick_params(axis='both', which='minor', left=False, labelsize=6)
    ax.set_aspect("auto")

    return im  # return image for colorbar

In [ ]:

n_plots = 12
alpha = 0.05

# ===== Layout labels =====
big_titles = ["Finger-tapping task", "Delayed centre-out reaching adaptation task"]
h_subtitles = ["Spectral power", "ITC", "Spectral power", "ITC"]
v_subtitles = ["Planning", "Execution", "Baseline", "Planning", "Execution", "Adaptation"]
v_sub_pos = [(2,1), (2,4), (slice(4,6),0), (4,1), (5,1), (slice(4,6),4)]

# ===== Figure setup =====
fig = plt.figure(figsize=(16, 8))
# Make GridSpec: 6 rows, 9 cols (last col for colorbar)
gs = GridSpec(6, 9, height_ratios=[0.1, 0.1, 1, 0.1, 1, 1], width_ratios=[0.05, 0.05, 1, 1, 0.05, 1, 1, 0.05, 0.05], figure=fig)

# # Debugging: visualize the grid layout
# for r in range(gs.nrows):
#     for c in range(gs.ncols):
#         ax = fig.add_subplot(gs[r, c])
#         ax.text(0.5, 0.5, f"{r},{c}", ha="center", va="center", fontsize=8)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         for spine in ax.spines.values():
#             spine.set_edgecolor("lightgray")

# Rows 1 and 4: Big titles spanning columns
title_indices = [0, 3]  # Row indices for big titles
for r_idx in title_indices:
    ax_title = fig.add_subplot(gs[r_idx, 2:7])
    ax_title.axis("off")
    title_idx = r_idx // 3  # 0 for first big title, 1 for second
    ax_title.text(0.5, 0.5, big_titles[title_idx], fontsize=18, ha="center", va="center", weight='bold')

# Row 2, columns 2, 3, 5, 6: Horizontal subtitles
h_col_indices = [2, 3, 5, 6]
for c_idx in h_col_indices:
    ax_subtitle = fig.add_subplot(gs[1, c_idx])
    ax_subtitle.axis("off")
    subtitle_idx = c_idx - 2 if c_idx < 5 else c_idx - 3
    ax_subtitle.text(0.5, 0.5, h_subtitles[subtitle_idx], fontsize=14, ha="center", va="center")

# Columns 1, 2, 5, 6: Vertical subtitles
for subtitle, pos in zip(v_subtitles, v_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center", rotation=90
    )


# TFR and ITC plots
plot_dict = {
    (2,2): ("Y_BL_plan", "power", (-4, 4)),
    (2,3): ("Y_BL_plan", "itc", (0, 0.2)),
    (2,5): ("Y_BL_go", "power", (-4, 4)),               
    (2,6): ("Y_BL_go", "itc", (0, 0.2)),
    (4,2): ("Y_MAIN_plan_baseline", "power", (-4, 4)),
    (4,3): ("Y_MAIN_plan_baseline", "itc", (0, 0.2)),
    (5,2): ("Y_MAIN_go_baseline", "power", (-4, 4)),
    (5,3): ("Y_MAIN_go_baseline", "itc", (0, 0.2)),
    (4,5): ("Y_MAIN_plan_adaptation", "power", (-4, 4)),
    (4,6): ("Y_MAIN_plan_adaptation", "itc", (0, 0.2)),
    (5,5): ("Y_MAIN_go_adaptation", "power", (-4, 4)),
    (5,6): ("Y_MAIN_go_adaptation", "itc", (0, 0.2))
    }

images = []
#read one power and one itc to get times and freqs
sample_sub = 's1_pac_sub01'
analysis_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y\\{sample_sub}\\preproc\\analysis'
power = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sample_sub}_BL_plan_power-tfr.h5"))
itc = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sample_sub}_BL_plan_itc-tfr.h5"))



for (r, c), (name, metric, clim) in plot_dict.items():
    ax = fig.add_subplot(gs[r, c])

    if metric == "power":
        # load power data (adjust filenames as needed)
        power_data = np.load(os.path.join(tfr_group_dir, f"TFR_CONCATENATED_{name}.npy"))
        power_data_ave = np.mean(power_data, axis=(0,1)) # average across subs and channels
        p_values_pow = np.load(os.path.join(tfr_group_dir, f"TFR_{name}_p_values.npy"))
        im = plot_tfr_stat(ax, power.times, power.freqs, power_data_ave,
                           p_values_pow, alpha, cmap="summer", clim=clim)
    elif metric == "itc":
        itc_data = np.load(os.path.join(tfr_group_dir, f"ITC_CONCATENATED_{name}.npy"))
        itc_data_ave = np.mean(itc_data, axis=(0,1)) # average across subs and channels
        p_values_itc = np.load(os.path.join(tfr_group_dir, f"ITC_{name}_p_values.npy"))
        im = plot_tfr_stat(ax, itc.times, itc.freqs, itc_data_ave,
                           p_values_itc, alpha, cmap="autumn", clim=clim)
    images.append(im)

# === Shared colorbars for both metrics ===
# Power
cax1 = fig.add_subplot(gs[2:6, 8])  # dedicate a vertical column for cbar
cbar1 = fig.colorbar(images[0], cax=cax1)
cbar1.set_label("Power (T-values)")

# ITC
cax2 = fig.add_subplot(gs[2:6, 7])
cbar2 = fig.colorbar(images[1], cax=cax2)
cbar2.set_label("ITC")

plt.tight_layout()
plt.savefig(os.path.join(paper1_figs_save_path, f"TFR_stats_{group}_all_conditions.png"), dpi=300)

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y\s1_pac_sub01\preproc\analysis\s1_pac_sub01_BL_plan_power-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y\s1_pac_sub01\preproc\analysis\s1_pac_sub01_BL_plan_itc-tfr.h5 ...
data shape: (40, 167), p_values shape: (40, 167)
times shape: (167,), freqs shape: (40,)
Masked data shape: (40, 167), significant mask shape: (40, 167)
data shape: (40, 167), p_values shape: (40, 167)
times shape: (167,), freqs shape: (40,)
Masked data shape: (40, 167), significant mask shape: (40, 167)
data shape: (40, 167), p_values shape: (40, 167)
times shape: (167,), freqs shape: (40,)
Masked data shape: (40, 167), significant mask shape: (40, 167)
data shape: (40, 167), p_values shape: (40, 167)
times shape: (167,), freqs shape: (40,)
Masked data shape: (40, 167), significant mask shape: (40, 167)
data shape: (40, 167), p_values shape: (40, 167)
times shape: (167,), freqs shape: (40,)
Masked data shape: (40, 167), significant m

# Figure 5
DONE

In [ ]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
pac_stats_save_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group\\pac_stats'
group = 'Y'

paper1_figs_save_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group\\paper1 figs'
check_paths(paper1_figs_save_path)

epochs_path = os.path.join(eeg_data_dir, group, 's1_pac_sub01', 'preproc', 'analysis', 's1_pac_sub01_BL_epochs_go-epo.fif')
epochs = mne.read_epochs(epochs_path, preload=True)
info = epochs.info
evoked = epochs.average()
evoked.data.shape

# n_plots = 12
# alpha = 0.05

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y\s1_pac_sub01\preproc\analysis\s1_pac_sub01_BL_epochs_go-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 10 columns
84 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [74]:
# maybe remove
def plot_significant_topomap(T_obs, clusters, cluster_p_values, info, p_thresh=0.05, vlim=None, ax=None,
                            #  group=None, task=None, task_stage=None, block_name=None
                             ):
    """
    Plots a topomap highlighting electrodes in significant clusters.

    Parameters
    ----------
    T_obs : array, shape (n_channels,)
        Observed T-values.
    clusters : list of boolean arrays
        Cluster masks (n_channels,).
    cluster_p_values : array
        P-values for each cluster.
    info : instance of mne.Info
        EEG info with channel locations.
    p_thresh : float
        Significance threshold.
    title : str
        Title for the plot.
    """
    # Combine significant cluster masks
    sig_mask = np.zeros_like(T_obs, dtype=bool)
    # for cluster, p_val in zip(clusters, cluster_p_values):
    #     if p_val <= p_thresh:
    #         sig_mask |= cluster

    # for cluster, p_val in zip(clusters, cluster_p_values):
    #     if p_val <= p_thresh:
    #         sig_mask |= cluster
    
    for cluster, p_val in zip(clusters, cluster_p_values):
        if p_val <= p_thresh:
            sig_mask |= cluster

    # Get color limits manually
    if vlim is None:
        vlim = np.nanmax(np.abs(T_obs))



    # Plot topomap with significant electrodes highlighted
    im, _ = mne.viz.plot_topomap(
    T_obs,
    info,
    cmap='PiYG',
    vlim=vlim,
    show=False,
    mask=sig_mask,
    mask_params=dict(marker='h', markersize=10, 
                markerfacecolor='y',
                markeredgecolor='k'),
    contours=0,
    axes=ax
    )

    return im


In [ ]:
# ===== Layout labels =====
big_titles = ["Finger-tapping task", "Delayed centre-out reaching adaptation task"]
h_subtitles = ["PAC MI", "Significant PAC", "PAC MI", "Significant PAC"]
v_subtitles = ["Planning", "Execution", "Baseline", "Planning", "Execution", "Adaptation"]
v_sub_pos = [(2,1), (2,4), (slice(4,6),0), (4,1), (5,1), (slice(4,6),4)]

# ===== Figure setup =====
fig = plt.figure(figsize=(16, 8))
# Make GridSpec: 6 rows, 9 cols (last col for colorbar)
gs = GridSpec(6, 9, height_ratios=[0.1, 0.1, 1, 0.1, 1, 1], width_ratios=[0.025, 0.025, 1, 1, 0.025, 1, 1, 0.05, 0.05], figure=fig)

# # Debugging: visualize the grid layout
# for r in range(gs.nrows):
#     for c in range(gs.ncols):
#         ax = fig.add_subplot(gs[r, c])
#         ax.text(0.5, 0.5, f"{r},{c}", ha="center", va="center", fontsize=8)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         for spine in ax.spines.values():
#             spine.set_edgecolor("lightgray")

# Rows 1 and 4: Big titles spanning columns
title_indices = [0, 3]  # Row indices for big titles
for r_idx in title_indices:
    ax_title = fig.add_subplot(gs[r_idx, 2:7])
    ax_title.axis("off")
    title_idx = r_idx // 3  # 0 for first big title, 1 for second
    ax_title.text(0.5, 0.5, big_titles[title_idx], fontsize=18, ha="center", va="center", weight='bold')

# Row 2, columns 2, 3, 5, 6: Horizontal subtitles
h_col_indices = [2, 3, 5, 6]
for c_idx in h_col_indices:
    ax_subtitle = fig.add_subplot(gs[1, c_idx])
    ax_subtitle.axis("off")
    subtitle_idx = c_idx - 2 if c_idx < 5 else c_idx - 3
    ax_subtitle.text(0.5, 0.5, h_subtitles[subtitle_idx], fontsize=14, ha="center", va="center")

# Columns 1, 2, 5, 6: Vertical subtitles
for subtitle, pos in zip(v_subtitles, v_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center", rotation=90
    )

# TFR and ITC plots
plot_dict = {
    (2,2): ("Y_BL_plan", "PAC", (0, 38e-5)),
    (2,3): ("Y_BL_plan", "Stats", (-4, 4)),
    (2,5): ("Y_BL_go", "PAC", (0, 38e-5)),
    (2,6): ("Y_BL_go", "Stats", (-4, 4)),
    (4,2): ("Y_MAIN_plan_baseline", "PAC", (0, 38e-5)),
    (4,3): ("Y_MAIN_plan_baseline", "Stats", (-4, 4)),
    (5,2): ("Y_MAIN_go_baseline", "PAC", (0, 38e-5)),
    (5,3): ("Y_MAIN_go_baseline", "Stats", (-4, 4)),
    (4,5): ("Y_MAIN_plan_adaptation", "PAC", (0, 38e-5)),
    (4,6): ("Y_MAIN_plan_adaptation", "Stats", (-4, 4)),
    (5,5): ("Y_MAIN_go_adaptation", "PAC", (0, 38e-5)),
    (5,6): ("Y_MAIN_go_adaptation", "Stats", (-4, 4))
    }

images = []

for (r, c), (name, metric, clim) in plot_dict.items():
    ax = fig.add_subplot(gs[r, c])

    if metric == "PAC":
        # load power data (adjust filenames as needed)
        pac_data = np.load(os.path.join(pac_stats_save_path, f"pac_mi_{name}_RAW.npy"))
        pac_all_ave = np.mean(pac_data, axis=(0, 2, 3))
        evoked.data = pac_all_ave
        im, _ = mne.viz.plot_topomap(
            evoked.data,
            evoked.info,
            ch_type="eeg",
            sensors=True,
            names=None,
            cmap="plasma",
            vlim=(clim[0], clim[1]),
            axes=ax,
            show=False
        )

    elif metric == "Stats":
        T_obs = np.load(os.path.join(pac_stats_save_path, f"pac_mi_{name}_freqs_ave_T_obs.npy"))
        clusters = np.load(os.path.join(pac_stats_save_path, f"pac_mi_{name}_freqs_ave_clusters.npy"), allow_pickle=True)
        clusters = [np.array(cl, dtype=bool) for cl in clusters]
        cluster_p_values = np.load(os.path.join(pac_stats_save_path, f"pac_mi_{name}_freqs_ave_cluster_p_values.npy"))


        im = plot_significant_topomap(T_obs, clusters, cluster_p_values, info, vlim=clim, ax=ax)

    images.append(im)

# === Shared colorbars for both metrics ===
# Power
cax1 = fig.add_subplot(gs[2:6, 8])  # dedicate a vertical column for cbar
cbar1 = fig.colorbar(images[0], cax=cax1)
cbar1.set_label("MI")

# ITC
cax2 = fig.add_subplot(gs[2:6, 7])
cbar2 = fig.colorbar(images[1], cax=cax2)
cbar2.set_label("T-values")

plt.tight_layout()
plt.savefig(os.path.join(paper1_figs_save_path, f"PAC_and_stats_{group}_sensors_all_conditions.png"), dpi=300)

# Figure 6


In [4]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'

# Load adjacency matrix
src_fname = 'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y\\fsaverage_bem\\bem\\fsaverage-ico4-src.fif'
src = mne.read_source_spaces(src_fname)
# src.plot(subjects_dir='D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y')
source_adjacency = mne.spatial_src_adjacency(src)
adj = source_adjacency.tocsr()  # Ensure adjacency is CSR format for fast indexing

# Load one stc file as a reference
stc_path = os.path.join('''D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y\\s1_pac_sub01\\preproc\\analysis\\source\\morphed_stcs\\_MAIN\\_plan\\_baseline\\s1_pac_sub01_MAIN_plan_baseline-stc-lcmv_epoch_000_morphed-lh.stc''')
stc_ref = mne.read_source_estimate(stc_path, subject='s1_pac_sub01') # any STC will serve as a template
stc_ref_mean = stc_ref.copy().mean()
stc_pac = stc_ref_mean.copy()
stc_pac.subject = "fsaverage_bem"

analysis_dir = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
source_group_dir = os.path.join(analysis_dir, 'sources')
pac_stats_save_path = os.path.join(analysis_dir, 'source_pac_stats')
paper1_figs_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\paper1 figs'
check_paths(source_group_dir, paper1_figs_save_path)

p_thresh = 0.05


    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124


In [ ]:
for task in tasks:
    if task == '_BL':
        block_names = ['']
    else:
        block_names = ['_baseline', '_adaptation']

    for task_stage in task_stages:
        for block_name in block_names:
            print(f'Processing {group} group, {task}{task_stage}{block_name}...')

            subs = os.listdir(os.path.join(eeg_data_dir, group))

            pac_list = []
            for sub_name in subs:
                analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
                pac_save_path = os.path.join(analysis_dir, 'source', 'PAC')
                pac_data = np.load(os.path.join(pac_save_path, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"))
                pac_data_t = np.transpose(pac_data, (1, 0, 2))

                ### NAN Imputation for PAC Matrices ###
                pac_imputed = pac_data_t.copy()

                # Step 1: Detect vertices with any NaN in their 20×20 PAC
                nan_mask = np.isnan(pac_data_t).any(axis=(1, 2))  # shape (5124,)

                if nan_mask.any() == True:
                    # print(f"Found {nan_mask.sum()} vertices with NaN values.")

                    # Step 2: Impute NaNs from neighbors
                    for vtx in np.where(nan_mask)[0]:
                        neighbors = adj[[vtx]].indices
                        valid_neighbors = [n for n in neighbors if not nan_mask[n]]
                        # Average PAC matrices from neighbors
                        pac_imputed[vtx] = np.nanmean(pac_data_t[valid_neighbors], axis=0)

                # pac_zscore = zscore(pac_imputed, axis=0, nan_policy='omit')
                pac_list.append(pac_imputed)

            # Stack them along a new first axis (subject axis)
            pac_all = np.stack(pac_list, axis=0)
            # print('PAC array shape:', pac_all.shape)
            pac_map = pac_all.mean(axis=(0, 2, 3))
            # print('Averaged PAC map shape:', pac_map.shape)
            stc_pac._data = pac_map[:, None]   # (n_vertices_total, 1)
            # print('STC data shape:', stc_pac.data.shape)
            # Save morphed stc
            stc_pac.save(os.path.join(source_group_dir, f"PAC_for_STATS_{group}{task}{task_stage}{block_name}_fsaverage"), overwrite=True)

In [12]:
# ===== Layout labels =====
big_title = ["Finger-tapping task"]
h_subtitles = ["PAC MI", "Significant PAC"]
v_subtitles = ["Planning", "Execution"]
v_sub_pos = [(slice(2,4),0), (slice(4,6),0)]
h_sub_pos = [(1, slice(1,3)), (1,3)]

# ===== Figure setup =====
fig = plt.figure(figsize=(14, 16))
# Make GridSpec: 6 rows, 9 cols (last col for colorbar)
gs = GridSpec(6, 6, height_ratios=[0.1, 0.1, 1, 1, 1, 1], width_ratios=[0.025, 1, 1, 1, 0.05, 0.05], figure=fig)

# # Debugging: visualize the grid layout
# for r in range(gs.nrows):
#     for c in range(gs.ncols):
#         ax = fig.add_subplot(gs[r, c])
#         ax.text(0.5, 0.5, f"{r},{c}", ha="center", va="center", fontsize=8)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         for spine in ax.spines.values():
#             spine.set_edgecolor("lightgray")

# Rows 1: Big title spanning columns
title_index = 0  # Row indices for big titles
ax_title = fig.add_subplot(gs[0, 0:4])
ax_title.axis("off")
ax_title.text(0.5, 0.5, big_title[title_index], fontsize=18, ha="center", va="center", weight='bold')

# Row 2, columns 2-3, 4: Horizontal subtitles
for subtitle, pos in zip(h_subtitles, h_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center"
    )

# Row 3-4, column 0: Vertical subtitles
for subtitle, pos in zip(v_subtitles, v_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center", rotation=90
    )

# MI and Stats source plots
low_lim = 1e-4 # 0.0001
top_lim = 20e-5 # 0.00025  # adjust based on your data
mean_lim = (low_lim + top_lim) / 2
stats_lims = [6.48e-05, (0.00025 + 6.48e-05) / 2, 0.00025]  # adjust based on your data
# Define plots with (row, col): (name, metric, projections, clims,
plot_dict = {
    (slice(2,4),1): ("Y_BL_plan", "PAC", ["lateral", "medial"], [low_lim, mean_lim, top_lim], 'figure'),
    (slice(2,4),2): ("Y_BL_plan", "PAC", ["dorsal", "ventral"], [low_lim, mean_lim, top_lim], 'figure'),
    (slice(2,4),3): ("Y_BL_plan", "Stats", ["dorsal", "lateral"], stats_lims, 'figure'),
    (slice(4,6),1): ("Y_BL_go", "PAC", ["lateral", "medial"], [low_lim, mean_lim, top_lim], 'figure'),
    (slice(4,6),2): ("Y_BL_go", "PAC", ["dorsal", "ventral"], [low_lim, mean_lim, top_lim], 'figure'),
    (slice(4,6),3): ("Y_BL_go", "Stats", None, None, 'text'),
    }

images = []

for (r, c), (name, metric, proj, clims, view) in plot_dict.items():

    clim = dict(
    kind="value",
    lims=clims,
    )

    ax = fig.add_subplot(gs[r, c])

    if view == 'figure':
        if metric == "PAC":

            stc_path = os.path.join(source_group_dir, f"PAC_for_STATS_{name}_fsaverage")
            stc_pac = mne.read_source_estimate(stc_path, subject='fsaverage_bem')

            # clim = dict(
            #     kind="value",
            #     lims=[
            #         np.nanmin(stc_pac.data), 
            #         (np.nanmax(stc_pac.data) - np.nanmin(stc_pac.data)) / 2 + np.nanmin(stc_pac.data), 
            #         np.nanmax(stc_pac.data)
            #         ],   # [min, mid, max]
            #     )
            # print(clim)

            lims = [(np.nanmin(stc_pac.data)), 
                    (np.nanmax(stc_pac.data) - np.nanmin(stc_pac.data)) / 2 + np.nanmin(stc_pac.data), 
                    np.nanmax(stc_pac.data)]
            clim = dict(
                kind="value",
                lims=lims
                )
            # print(clim)

            cax1 = fig.add_subplot(gs[r, 4])  # dedicate a vertical column for cbar
            # choose the colormap you used in your brain plot
            cmap = plt.cm.plasma  
            # create norm with 3 boundaries
            norm = mpl.colors.Normalize(vmin=lims[0], vmax=lims[2])
            # create ScalarMappable
            sm = ScalarMappable(norm=norm, cmap=cmap)
            # plot colorbar
            cbar = fig.colorbar(sm, cax=cax1)
            cbar.set_label("MI", fontsize=10)
            cbar.ax.tick_params(labelsize=6) 
            

            brain = stc_pac.plot(subject="fsaverage_bem",
                        initial_time=0.0,
                        clim=clim,
                        cortex="low_contrast",
                        colormap='plasma', # 'plasma' for raw MI, 'cool' for zscored MI
                        hemi='both',
                        views=proj,
                        background='white',
                        colorbar=False,
                        )

            # take screenshot as numpy array
            screenshot = brain.screenshot()

            # show in your Matplotlib subplot
            ax.imshow(screenshot)
            ax.axis("off") 
        
        else:
            
            stc_path = os.path.join(source_group_dir, f"PAC_for_STATS_{name}_fsaverage")
            stc_pac = mne.read_source_estimate(stc_path, subject='fsaverage_bem')

            # Load stats data
            T_obs = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_T_obs.npy"))
            clusters = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_clusters.npy"), allow_pickle=True)
            cluster_p_values = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_cluster_p_values.npy"))

            # Combine significant cluster masks
            sig_mask = np.zeros_like(T_obs, dtype=bool)
            for cluster, p_val in zip(clusters, cluster_p_values):
                if p_val <= p_thresh:
                    sig_mask |= cluster.astype(bool)
            sig_mask = sig_mask[:, None]

            # Copy the STC to avoid modifying the original
            stc_sig = copy.deepcopy(stc_pac)
            stc_sig.data[~sig_mask] = 0.0

            # view = [["dorsal", "lateral"] if sig_cond == 'Y_BL_plan_' else ["dorsal", "parietal"]]
            brain = stc_sig.plot(subject="fsaverage_bem",
                        initial_time=0.0,
                        clim=clim,
                        cortex="low_contrast",
                        colormap='hot', # 'plasma' for raw MI, 'cool' for zscored MI
                        hemi='both',
                        views=proj,
                        colorbar=False,
                        background='white',
                        )

            # take screenshot as numpy array
            screenshot = brain.screenshot()

            # show in your Matplotlib subplot
            ax.imshow(screenshot)
            ax.axis("off") 

    else:
        # Just add text
        ax.text(0.5, 0.5, "No significant \nclusters", ha="center", va="center", fontsize=12)
        ax.axis("off")
    
    images.append(brain)

# # === Shared colorbars for both metrics ===
# # MI
# cax1 = fig.add_subplot(gs[2:6, 4])  # dedicate a vertical column for cbar
# lims = [low_lim, mean_lim, top_lim]
# # choose the colormap you used in your brain plot
# cmap = plt.cm.plasma  
# # create norm with 3 boundaries
# norm = mpl.colors.Normalize(vmin=low_lim, vmax=top_lim)
# # create ScalarMappable
# sm = ScalarMappable(norm=norm, cmap=cmap)
# # plot colorbar
# cbar = fig.colorbar(sm, cax=cax1)
# cbar.set_label("MI", fontsize=10)
# cbar.ax.tick_params(labelsize=6) 

# # Stats
cax2 = fig.add_subplot(gs[2:4, 5])  # dedicate a vertical column for cbar
lims = stats_lims
# choose the colormap you used in your brain plot
cmap = plt.cm.hot
# create norm with 3 boundaries
norm = mpl.colors.Normalize(vmin=lims[0], vmax=lims[2])
# create ScalarMappable
sm = ScalarMappable(norm=norm, cmap=cmap)
# plot colorbar
cbar = fig.colorbar(sm, cax=cax2)
cbar.set_label("T-values", fontsize=10)
cbar.ax.tick_params(labelsize=6) 

plt.tight_layout()

# SAVING OPTIONS
# Better go to full screen and save MANUALLY from there to get better resolution
# fig.savefig(os.path.join(paper1_figs_save_path, f"PAC_and_stats_{group}_source_FTT_test.png"), dpi=300)


# Figure 7

STOPPED AT clims FOR PLOTS !!!!

In [ ]:
{'kind': 'value', 'lims': [np.float32(0.00024879884), np.float32(0.00029916066), np.float32(0.00034952245)]}
{'kind': 'value', 'lims': [np.float32(0.00024879884), np.float32(0.00029916066), np.float32(0.00034952245)]}
{'kind': 'value', 'lims': [np.float32(0.00014511288), np.float32(0.00017538367), np.float32(0.00020565446)]}
{'kind': 'value', 'lims': [np.float32(0.00014511288), np.float32(0.00017538367), np.float32(0.00020565446)]}
{'kind': 'value', 'lims': [np.float32(9.372515e-05), np.float32(0.00012279808), np.float32(0.000151871)]}
{'kind': 'value', 'lims': [np.float32(9.372515e-05), np.float32(0.00012279808), np.float32(0.000151871)]}
{'kind': 'value', 'lims': [np.float32(5.152758e-05), np.float32(6.4537795e-05), np.float32(7.7548015e-05)]}
{'kind': 'value', 'lims': [np.float32(5.152758e-05), np.float32(6.4537795e-05), np.float32(7.7548015e-05)]}

In [8]:
np.format_float_positional(lims[0])

'0.0000648'

In [ ]:
# ===== Layout labels =====
big_title = ["Delayed centre-out reaching adaptation task"]
h_subtitles = ["PAC MI", "Significant PAC"]
v_subtitles = ["Baseline", "Adaptation", "Planning", "Execution", "Planning", "Execution"]
v_sub_pos = [(slice(2,6),0), (slice(6,10),0), (slice(2,4),1), (slice(4,6),1), (slice(6,8),1), (slice(8,10),1)]
h_sub_pos = [(1, slice(1,3)), (1,3)]

# ===== Figure setup =====
fig = plt.figure(figsize=(14, 16))
# Make GridSpec: 9 rows, 7 cols (last col for colorbar)
gs = GridSpec(10, 7, height_ratios=[0.1, 0.5, 1, 1, 1, 1, 1, 1, 1, 1], width_ratios=[0.025, 0.025, 1, 1, 1, 0.05, 0.05], figure=fig)

# # Debugging: visualize the grid layout
# for r in range(gs.nrows):
#     for c in range(gs.ncols):
#         ax = fig.add_subplot(gs[r, c])
#         ax.text(0.5, 0.5, f"{r},{c}", ha="center", va="center", fontsize=8)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         for spine in ax.spines.values():
#             spine.set_edgecolor("lightgray")

# Rows 1: Big title spanning columns
title_index = 0  # Row indices for big titles
ax_title = fig.add_subplot(gs[0, :])
ax_title.axis("off")
ax_title.text(0.5, 0.5, big_title[title_index], fontsize=18, ha="center", va="center", weight='bold')

# Row 2, columns 2-3, 4: Horizontal subtitles
for subtitle, pos in zip(h_subtitles, h_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center"
    )

# Vertical subtitles
for subtitle, pos in zip(v_subtitles, v_sub_pos):
    ax_subtitle = fig.add_subplot(gs[pos])  # works with both tuples and slices
    ax_subtitle.axis("off")
    ax_subtitle.text(
        0.5, 0.5, subtitle,
        fontsize=14, ha="center", va="center", rotation=90
    )

# MI and Stats source plots
# low_lim = 1e-10 # 1e-10 # 0.0001
# top_lim = 50e-5 # 0.00025  # adjust based on your data
# mean_lim = (low_lim + top_lim) / 2
stats_lims = [6.48e-05, (0.00025 + 6.48e-05) / 2, 0.00025] 
# Define plots with (row, col): (name, metric, projections, clims,
plot_dict = {
    (slice(2,4),2): ("Y_MAIN_plan_baseline", "PAC", ["lateral", "medial"], None, 'figure'),
    (slice(2,4),3): ("Y_MAIN_plan_baseline", "PAC", ["dorsal", "ventral"], None, 'figure'),
    (slice(2,4),4): ("Y_MAIN_plan_baseline", "Stats", ["dorsal", "parietal"], stats_lims, 'figure'),
    (slice(4,6),2): ("Y_MAIN_go_baseline", "PAC", ["lateral", "medial"], None, 'figure'),
    (slice(4,6),3): ("Y_MAIN_go_baseline", "PAC", ["dorsal", "ventral"], None, 'figure'),
    (slice(4,6),4): ("Y_MAIN_go_baseline", "Stats", None, None, 'text'),
    (slice(6,8),2): ("Y_MAIN_plan_adaptation", "PAC", ["lateral", "medial"], None, 'figure'),
    (slice(6,8),3): ("Y_MAIN_plan_adaptation", "PAC", ["dorsal", "ventral"], None, 'figure'),
    (slice(6,8),4): ("Y_MAIN_plan_adaptation", "Stats", None, None, 'text'),
    (slice(8,10),2): ("Y_MAIN_go_adaptation", "PAC", ["lateral", "medial"], None, 'figure'),
    (slice(8,10),3): ("Y_MAIN_go_adaptation", "PAC", ["dorsal", "ventral"], None, 'figure'),
    (slice(8,10),4): ("Y_MAIN_go_adaptation", "Stats", None, None, 'text')
    }

images = []

for (r, c), (name, metric, proj, clims, view) in plot_dict.items():

    ax = fig.add_subplot(gs[r, c])

    if view == 'figure':
        if metric == "PAC":

            stc_path = os.path.join(source_group_dir, f"PAC_for_STATS_{name}_fsaverage")
            stc_pac = mne.read_source_estimate(stc_path, subject='fsaverage_bem')

            lims = [(np.nanmin(stc_pac.data)), 
                    (np.nanmax(stc_pac.data) - np.nanmin(stc_pac.data)) / 2 + np.nanmin(stc_pac.data), 
                    np.nanmax(stc_pac.data)]
            clim = dict(
                kind="value",
                lims=lims
                )
            # print(clim)

            cax1 = fig.add_subplot(gs[r, 5])  # dedicate a vertical column for cbar
            # choose the colormap you used in your brain plot
            cmap = plt.cm.plasma  
            # create norm with 3 boundaries
            norm = mpl.colors.Normalize(vmin=lims[0], vmax=lims[2])
            # create ScalarMappable
            sm = ScalarMappable(norm=norm, cmap=cmap)
            # plot colorbar
            cbar = fig.colorbar(sm, cax=cax1)
            cbar.set_label("MI", fontsize=8)
            cbar.ax.tick_params(labelsize=6) 

            brain = stc_pac.plot(subject="fsaverage_bem",
                        initial_time=0.0,
                        clim=clim,
                        cortex="low_contrast",
                        colormap='plasma', # 'plasma' for raw MI, 'cool' for zscored MI
                        hemi='both',
                        views=proj,
                        background='white',
                        colorbar=False,
                        )

            # take screenshot as numpy array
            screenshot = brain.screenshot()

            # show in your Matplotlib subplot
            ax.imshow(screenshot)
            ax.axis("off") 
        
        else:
            
            stc_path = os.path.join(source_group_dir, f"PAC_for_STATS_{name}_fsaverage")
            stc_pac = mne.read_source_estimate(stc_path, subject='fsaverage_bem')

            # Load stats data
            T_obs = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_T_obs.npy"))
            clusters = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_clusters.npy"), allow_pickle=True)
            cluster_p_values = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{name}_freqs_ave_cluster_p_values.npy"))

            # Combine significant cluster masks
            sig_mask = np.zeros_like(T_obs, dtype=bool)
            for cluster, p_val in zip(clusters, cluster_p_values):
                if p_val <= p_thresh:
                    sig_mask |= cluster.astype(bool)
            sig_mask = sig_mask[:, None]

            # Copy the STC to avoid modifying the original
            stc_sig = copy.deepcopy(stc_pac)
            stc_sig.data[~sig_mask] = 0.0

            # view = [["dorsal", "lateral"] if sig_cond == 'Y_BL_plan_' else ["dorsal", "parietal"]]
            brain = stc_sig.plot(subject="fsaverage_bem",
                        initial_time=0.0,
                        clim=clim,
                        cortex="low_contrast",
                        colormap='hot', # 'plasma' for raw MI, 'cool' for zscored MI
                        hemi='both',
                        views=proj,
                        colorbar=False,
                        background='white',
                        )

            # take screenshot as numpy array
            screenshot = brain.screenshot()

            # show in your Matplotlib subplot
            ax.imshow(screenshot)
            ax.axis("off") 

    else:
        # Just add text
        ax.text(0.5, 0.5, "No significant \nclusters", ha="center", va="center", fontsize=12)
        ax.axis("off")
    
    images.append(brain)

# === Shared colorbars for both metrics ===
# MI
# cax1 = fig.add_subplot(gs[2:10, 5])  # dedicate a vertical column for cbar
# lims = [low_lim, mean_lim, top_lim]
# # choose the colormap you used in your brain plot
# cmap = plt.cm.plasma  
# # create norm with 3 boundaries
# norm = mpl.colors.Normalize(vmin=low_lim, vmax=top_lim)
# # create ScalarMappable
# sm = ScalarMappable(norm=norm, cmap=cmap)
# # plot colorbar
# cbar = fig.colorbar(sm, cax=cax1)
# cbar.set_label("MI", fontsize=10)
# cbar.ax.tick_params(labelsize=6) 

# # Stats
cax2 = fig.add_subplot(gs[2:4, 6])  # dedicate a vertical column for cbar
lims = [6.48e-05, (0.00025 + 6.48e-05) / 2, 0.00025]
# choose the colormap you used in your brain plot
cmap = plt.cm.hot
# create norm with 3 boundaries
norm = mpl.colors.Normalize(vmin=lims[0], vmax=lims[2])
# create ScalarMappable
sm = ScalarMappable(norm=norm, cmap=cmap)
# plot colorbar
cbar = fig.colorbar(sm, cax=cax2)
cbar.set_label("T-values", fontsize=10)
cbar.ax.tick_params(labelsize=6) 

plt.tight_layout()

# SAVING OPTIONS
# Better go to full screen and save MANUALLY from there to get better resolution
fig.savefig(os.path.join(paper1_figs_save_path, f"PAC_and_stats_{group}_source_DeCRAT_test.png"), dpi=300)


# Figure 8
DONE

In [2]:
pac_stats_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group\\pac_stats'
pac_stats_path = os.path.join(pac_stats_dir, 'conditions')

group = 'Y'
task = '_MAIN'
task_stage = '_plan'
block_name = '_baseline' # '_baseline', '_adaptation'
sub_name = 's1_pac_sub01'

fig_group_path = os.path.join(pac_stats_dir, 'figs')
fig_group_save_path = os.path.join(fig_group_path, 'paper1')

# Load EEG data
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
epochs_path = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
epochs = mne.read_epochs(os.path.join(epochs_path, f"{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif"), preload=True)
eeg_channel_names = epochs.copy().pick("eeg").ch_names
epochs.pick(eeg_channel_names)
info = epochs.info
n_channels = len(info.ch_names)
n_plots = 6

# Leave only six conditions for plotting
comparisons = [
    ("MAIN_plan_combined", "BL_plan"),
    ("MAIN_go_combined", "BL_go"),
]


Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y\s1_pac_sub01\preproc\analysis\s1_pac_sub01_MAIN_epochs_plan_baseline-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 15 columns
50 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [ ]:
T_obs_data = []
clusters_data = []
cluster_p_values_data = []

for comparison in comparisons:
    cond1 = comparison[0]
    cond2 = comparison[1]

    # # Save the results
    T_obs = np.load(os.path.join(pac_stats_path, f"pac_mi_{group}_{cond1}_vs_{cond2}_freqs_ave_T_obs.npy"))
    clusters = np.load(os.path.join(pac_stats_path, f"pac_mi_{group}_{cond1}_vs_{cond2}_freqs_ave_clusters.npy"), allow_pickle=True)
    clusters = np.asarray(clusters, dtype=bool)
    cluster_p_values = np.load(os.path.join(pac_stats_path, f"pac_mi_{group}_{cond1}_vs_{cond2}_freqs_ave_cluster_p_values.npy"))
    T_obs_data.append(T_obs)
    clusters_data.append(clusters)
    cluster_p_values_data.append(cluster_p_values)

n_plots = 2
p_thresh = 0.05

# ===== Layout labels =====
big_title = ["FTT vs De-CRAT"]
condition_labels = ["Planning", "Execution"]

# ===== Figure setup =====
fig = plt.figure(figsize=(9, 5))
# Make GridSpec: 3 rows, 3 cols (last col for colorbar)
gs = GridSpec(3, 3, height_ratios=[0.1, 0.1, 1], width_ratios=[1, 1, 0.05], figure=fig)

# Row 1: Big titles spanning columns
ax_title1 = fig.add_subplot(gs[0, 0:2])
ax_title1.axis("off")
ax_title1.text(0.5, 0.5, big_title[0], fontsize=18, ha="center", va="center", weight='bold')

# Empty placeholder to align "Planning" and "Execution"
for i in range(3, 2):
    ax = fig.add_subplot(gs[1, i])
    ax.axis("off")

# Row 2: Condition labels
labels = condition_labels
for i, label in enumerate(labels):
    ax = fig.add_subplot(gs[1, i])
    ax.axis("off")
    ax.text(0.5, 0.5, label, fontsize=14, ha="center", va="center", weight='semibold')

# # Row 3: Topoplots
# Topoplots
for idx in range(n_plots):
    # Combine significant cluster masks
    sig_mask = np.zeros_like(T_obs_data[idx], dtype=bool)
    for cluster, p_val in zip(clusters_data[idx], cluster_p_values_data[idx]):
        if p_val <= p_thresh:
            sig_mask |= cluster

    ax = fig.add_subplot(gs[2, idx])
    im, _ = mne.viz.plot_topomap(
        T_obs_data[idx], info, axes=ax,
        show=False, cmap="PiYG", vlim=(-4, 4),
        contours=0, mask=sig_mask,
        mask_params=dict(marker='h', markersize=15,
                         markerfacecolor='y', markeredgecolor='k'),
        sensors=True
    )

# Colorbar in its own column
cax = fig.add_subplot(gs[2, 2])  
cbar = fig.colorbar(im, cax=cax)
cbar.set_label("T-value", fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.05, wspace=0.01)  # move topoplots up, reduce vertical gaps
fig.canvas.draw()  # update positions

plt.savefig(os.path.join(paper1_figs_save_path, f'fig_topo_{group}_{cond1}_vs_{cond2}.png'), dpi=300)

__________________________________

In [ ]:
import matplotlib.pyplot as plt
import mne
import numpy as np
from matplotlib.gridspec import GridSpec

# ===== Dummy EEG info =====
# drop duplicate aliases that cause overlap
bad_chs = ['T3', 'T4', 'T5', 'T6']  # old labels that duplicate modern ones
montage = mne.channels.make_standard_montage("standard_1020")
info = mne.create_info(ch_names=montage.ch_names, sfreq=1000, ch_types="eeg")
info = mne.pick_info(info, sel=mne.pick_channels(info.ch_names,
                                                 [ch for ch in info.ch_names if ch not in bad_chs]))
info.set_montage(montage)

# ===== Dummy EEG data (replace with your T_obs etc.) =====
n_channels = len(info.ch_names)  # <- matches info exactly
n_plots = 2
fake_data = [np.random.randn(n_channels) for _ in range(n_plots)]

# ===== Layout labels =====
big_title = ["FTT vs De-CRAT"]
# task_subtitles = ["FTT", "De-CRAT"]
# condition_labels = ["Baseline", "Adaptation", "Combined", "Planning", "Execution"]
condition_labels = ["Planning", "Execution"]

# ===== Figure setup =====
fig = plt.figure(figsize=(16, 5))
gs = GridSpec(3, 2, height_ratios=[0.1, 0.1, 1], figure=fig)

# Row 1: Big titles spanning columns
ax_title1 = fig.add_subplot(gs[0, 0:2])
ax_title1.axis("off")
ax_title1.text(0.5, 0.5, big_titles[0], fontsize=18, ha="center", va="center", weight='bold')

# Empty placeholder to align "Planning" and "Execution"
for i in range(3, 2):
    ax = fig.add_subplot(gs[1, i])
    ax.axis("off")

# Row 2: Condition labels
labels = condition_labels
for i, label in enumerate(labels):
    ax = fig.add_subplot(gs[1, i])
    ax.axis("off")
    ax.text(0.5, 0.5, label, fontsize=12, ha="center", va="center", weight='bold')

# Row 3: Topoplots
for idx in range(n_plots):
    ax = fig.add_subplot(gs[2, idx])
    mne.viz.plot_topomap(
        fake_data[idx], info, axes=ax,
        show=False, cmap="PiYG", vlim=(-4, 4),
        contours=0,
        # Example of significance markers
        mask=np.random.choice([False, True], size=n_channels, p=[0.9, 0.1]),
        mask_params=dict(marker='o', markersize=8, markerfacecolor='yellow')
    )

plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.01)  # move topoplots up, reduce vertical gaps
fig.canvas.draw()  # update positions


In [20]:
fake_data[0]

array([ 0.31039847, -2.76542652, -1.02771569, -0.59622677, -0.81083374,
       -0.41914056,  1.16983676, -0.69712082,  0.42725258, -0.04392646,
       -0.31735741, -0.67280614,  0.47051627,  0.48367262, -0.7596814 ,
        0.88309154, -0.51894878,  0.07367918,  0.32037646, -0.40570262,
        1.37618077, -0.39809528,  0.53725099, -1.25458377,  0.53951271,
        1.40934169,  0.66957183, -0.04308962,  0.88362252,  2.03874478,
        1.03828873,  0.41367676, -0.08934078,  1.71624308,  0.21930756,
        0.2418728 ,  1.28742938,  0.44250171, -0.88016663, -0.4175251 ,
        0.6829618 ,  0.68893056, -1.05727155,  1.4101829 , -0.19366157,
        0.81007349, -1.5162131 ,  0.02936543, -0.69743416, -1.14917709,
        0.44966921,  1.0478121 ,  0.13783028, -0.6749554 , -0.74217064,
        0.95988297, -0.44704385, -0.49624332,  0.11237568, -0.76335137,
       -0.4608839 , -0.07258125, -1.86366018, -0.06587098, -0.52618407,
       -1.46500592, -1.73851083, -0.56282069, -0.00798849,  2.62

______________________________

In [ ]:
import matplotlib.pyplot as plt
import mne
import numpy as np
from matplotlib.gridspec import GridSpec

# ===== Dummy EEG info =====
# drop duplicate aliases that cause overlap
bad_chs = ['T3', 'T4', 'T5', 'T6']  # old labels that duplicate modern ones
montage = mne.channels.make_standard_montage("standard_1020")
info = mne.create_info(ch_names=montage.ch_names, sfreq=1000, ch_types="eeg")
info = mne.pick_info(info, sel=mne.pick_channels(info.ch_names,
                                                 [ch for ch in info.ch_names if ch not in bad_chs]))
info.set_montage(montage)

# ===== Dummy EEG data (replace with your T_obs etc.) =====
n_channels = len(info.ch_names)  # <- matches info exactly
n_plots = 6
fake_data = [np.random.randn(n_channels) for _ in range(n_plots)]

# ===== Layout labels =====
big_titles = ["Execution vs Planning", "FTT vs De-CRAT"]
task_subtitles = ["FTT", "De-CRAT"]
condition_labels = ["Baseline", "Adaptation", "Combined", "Planning", "Execution"]

# ===== Figure setup =====
fig = plt.figure(figsize=(16, 5))
gs = GridSpec(4, 6, height_ratios=[0.1, 0.1, 0.1, 1], figure=fig)

# Row 1: Big titles spanning columns
ax_title1 = fig.add_subplot(gs[0, 0:4])
ax_title1.axis("off")
ax_title1.text(0.5, 0.5, big_titles[0], fontsize=18, ha="center", va="center")

ax_title2 = fig.add_subplot(gs[0, 4:6])
ax_title2.axis("off")
ax_title2.text(0.5, 0.5, big_titles[1], fontsize=18, ha="center", va="center")

# Row 2: Task subtitles
ax_sub1 = fig.add_subplot(gs[1, 0])
ax_sub1.axis("off")
ax_sub1.text(0.5, 0.5, task_subtitles[0], fontsize=14, ha="center", va="center")

ax_sub2 = fig.add_subplot(gs[1, 1:4])
ax_sub2.axis("off")
ax_sub2.text(0.5, 0.5, task_subtitles[1], fontsize=14, ha="center", va="center")

# Empty placeholder to align "Planning" and "Execution"
for i in range(3, 6):
    ax = fig.add_subplot(gs[1, i])
    ax.axis("off")

# Row 3: Condition labels
labels = [" "] + condition_labels
for i, label in enumerate(labels):
    ax = fig.add_subplot(gs[2, i])
    ax.axis("off")
    ax.text(0.5, 0.5, label, fontsize=12, ha="center", va="center")

# Row 4: Topoplots
for idx in range(n_plots):
    ax = fig.add_subplot(gs[3, idx])
    mne.viz.plot_topomap(
        fake_data[idx], info, axes=ax,
        show=False, cmap="PiYG", vlim=(-4, 4),
        contours=0,
        # Example of significance markers
        mask=np.random.choice([False, True], size=n_channels, p=[0.9, 0.1]),
        mask_params=dict(marker='o', markersize=8, markerfacecolor='yellow')
    )

plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.01)  # move topoplots up, reduce vertical gaps
fig.canvas.draw()  # update positions

for i, ax in enumerate(fig.axes[-n_plots:]):
    if i == 1:
        note = "*not significant"
        bbox = ax.get_position()
        x = bbox.x0 + bbox.width / 2
        y = bbox.y0 - 0.05
        fig.text(x, y, f"{note}", ha="center", va="top", fontsize=10, color="red")



In [16]:
n_plots

6